# Final Project-Group 5 Data Science Class

## Step 0: Dataset Description

The dataset gives a detailed view of sales transactions, including customer demographics (age, gender, location), product details, and financial metrics like cost, revenue, and profit. It supports analysis of purchasing behavior across demographic groups and trends over time based on product quantity sold. Containing both numeric and categorical data, it enables multi-level analysis by product, customer, and region using various visualization and statistical techniques. It has 13 columns and 34865 rows. 

## Step 1: Data Pipeline & Preparation

### 1.1. Data Acquisition

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from statsmodels.tsa.arima.model import ARIMA

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [8]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(r"C:\Users\hp probook\Desktop\data_science_group5\Group 5-Dataset for Final Project.csv")
df.head()
print(df)

       index       Date    Year     Month  Customer Age Customer Gender        Country           State Product Category     Sub Category  Quantity  Unit Cost   Unit Price    Cost      Revenue
0          0  2/19/2016  2016.0  February          29.0               F  United States      Washington      Accessories  Tires and Tubes       1.0      80.00   109.000000    80.0   109.000000
1          1  2/20/2016  2016.0  February          29.0               F  United States      Washington         Clothing           Gloves       2.0      24.50    28.500000    49.0    57.000000
2          2  2/27/2016  2016.0  February          29.0               F  United States      Washington      Accessories  Tires and Tubes       3.0       3.67     5.000000    11.0    15.000000
3          3  3/12/2016  2016.0     March          29.0               F  United States      Washington      Accessories  Tires and Tubes       2.0      87.50   116.500000   175.0   233.000000
4          4  3/12/2016  2016.0     Marc

### 1.2 Check Data Types

In [9]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34867 entries, 0 to 34866
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             34867 non-null  int64  
 1   Date              34866 non-null  object 
 2   Year              34866 non-null  float64
 3   Month             34866 non-null  object 
 4   Customer Age      34716 non-null  float64
 5   Customer Gender   34766 non-null  object 
 6   Country           34746 non-null  object 
 7   State             34866 non-null  object 
 8   Product Category  34866 non-null  object 
 9   Sub Category      34866 non-null  object 
 10  Quantity          34666 non-null  float64
 11  Unit Cost         34686 non-null  float64
 12  Unit Price        34706 non-null  float64
 13  Cost              34866 non-null  float64
 14  Revenue           34567 non-null  float64
dtypes: float64(7), int64(1), object(7)
memory usage: 4.0+ MB
None
index                 0
D

### 1.3. Handle Duplicates & Outliers

In [13]:
df.drop_duplicates(inplace=True)

for col in ['Quantity', 'Unit Cost', 'Unit Price', 'Revenue']:
    q_low = df[col].quantile(0.01)
    q_hi  = df[col].quantile(0.99)
    df = df[(df[col] >= q_low) & (df[col] <= q_hi)]
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 30496 entries, 0 to 34864
Data columns (total 76 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   index                           30496 non-null  int64         
 1   Date                            30496 non-null  datetime64[ns]
 2   Year                            30496 non-null  float64       
 3   Month                           30496 non-null  object        
 4   Customer Age                    30368 non-null  float64       
 5   Quantity                        30496 non-null  float64       
 6   Unit Cost                       30496 non-null  float64       
 7   Unit Price                      30496 non-null  float64       
 8   Cost                            30496 non-null  float64       
 9   Revenue                         30496 non-null  float64       
 10  Customer Gender_M               30496 non-null  bool          
 11  Country

### 1.4 Fill the Missing Values

In [18]:
# Fill the missing values with the mean
df['Customer Age'] = df['Customer Age'].fillna(df['Customer Age'].mean())

### 1.5 Data Preparation Final Check

In [22]:
# 1. Check for Missing Values
print("Missing values in each column:")
print(df.isnull().sum())
print("\n")  # Add a new line for clarity

# 2. Check Data Types
print("Data types of each column:")
print(df.dtypes)
print("\n")

# 3. Summary Statistics
print("Summary statistics of the numerical columns:")
print(df.describe())
print("\n")

# 4. Check for Duplicates
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")
print("\n")

# 5. Validate Categorical Columns (Example: Check 'Month' and 'Customer Gender_M')
print("Unique values in 'Month' column:")
print(df['Month'].unique())
print("\n")

print("Unique values in 'Customer Gender_M' column:")
print(df['Customer Gender_M'].unique())
print("\n")

# 6. Final Review of Data Integrity (Show brief summary of the first few rows)
print("Dataset shape (rows, columns):")
print(df.shape)
print("\n")

print("Summary of the first few rows (column names, non-null counts, data types):")
print(df.info())


Missing values in each column:
index                           0
Date                            0
Year                            0
Month                           0
Customer Age                    0
                               ..
Sub Category_Shorts             0
Sub Category_Socks              0
Sub Category_Tires and Tubes    0
Sub Category_Touring Bikes      0
Sub Category_Vests              0
Length: 76, dtype: int64


Data types of each column:
index                                    int64
Date                            datetime64[ns]
Year                                   float64
Month                                   object
Customer Age                           float64
                                     ...      
Sub Category_Shorts                       bool
Sub Category_Socks                        bool
Sub Category_Tires and Tubes              bool
Sub Category_Touring Bikes                bool
Sub Category_Vests                        bool
Length: 76, dtype: objec

#### Data Preparation Narrative Conclusion

Based on the above critical checks for missing values, data types, duplicates, and column cleanliness, our dataset looks clean and ready for analysis. The following will be the exploration data analysis. 

## Step 2: Exploratory Data Analysis (EDA)

### 2.1 Descriptive Statistics